# Build Final Exam Data

In [1]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
from binomial import *
from ratecurves import *
from treasury_cmds import compound_rate
from volskew import *

import datetime

In [2]:
DATE = '2024-03-05'

In [3]:
SAVEDATA = True
FILEOUT = f'../data/exam_data_{DATE}.xlsx'

# Curves

In [4]:
freqcurve = 4
curves = pd.read_excel(f'../data/cap_curves_{DATE}.xlsx', sheet_name=f'rate curves {DATE}').set_index('tenor')

### Limit maturity

In [5]:
LIMIT = 3
curves = curves.loc[:LIMIT]

In [6]:
curves.style.format('{:.2%}').format_index('{:.2f}')

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,5.22%,5.22%,98.71%,nan%,nan%,nan%
0.50,5.15%,5.15%,97.48%,5.07%,14.05%,14.05%
0.75,5.04%,5.04%,96.31%,4.83%,16.19%,17.86%
1.00,4.92%,4.92%,95.23%,4.56%,18.33%,21.53%
1.25,4.75%,4.75%,94.27%,4.05%,23.38%,35.39%
1.50,4.61%,4.60%,93.37%,3.88%,27.29%,37.77%
1.75,4.49%,4.48%,92.50%,3.75%,30.22%,39.36%
2.00,4.39%,4.38%,91.66%,3.66%,32.33%,39.89%
2.25,4.29%,4.27%,90.88%,3.44%,33.77%,39.50%


# BDT

In [7]:
sigmas = curves['fwd vols']
sigmas.iloc[0] = sigmas.iloc[1]
sigmas.index = sigmas.index.to_numpy().round(6)

theta, ratetree = estimate_theta(sigmas,100*curves.loc[:,'discounts'])
format_bintree(ratetree,style='{:.2%}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,5.19%,5.39%,5.54%,5.69%,5.57%,5.94%,6.49%,7.25%,7.87%,9.23%,10.98%,13.04%
1,,4.69%,4.82%,4.95%,4.84%,5.16%,5.64%,6.30%,6.83%,8.02%,9.54%,11.33%
2,,,4.03%,4.14%,4.05%,4.32%,4.71%,5.27%,5.72%,6.71%,7.98%,9.47%
3,,,,3.34%,3.26%,3.48%,3.80%,4.25%,4.61%,5.41%,6.43%,7.64%
4,,,,,2.29%,2.44%,2.67%,2.98%,3.24%,3.80%,4.52%,5.36%
5,,,,,,1.68%,1.83%,2.04%,2.22%,2.60%,3.10%,3.68%
6,,,,,,,1.23%,1.38%,1.50%,1.76%,2.09%,2.48%
7,,,,,,,,0.93%,1.00%,1.18%,1.40%,1.66%
8,,,,,,,,,0.68%,0.79%,0.94%,1.12%


# Vol Skew

These are input manually from Bloomberg's `VCUB` dashboard.

In [8]:
voldata = pd.read_excel('../data/swaption_vol_data_registry.xlsx')
voldata

,reference,instrument,model,date,expiration,tenor,-300,-200,-100,-50,-25,0,25,50,100,200,300
0,SOFR,swaption,black,2024-03-05,2.00,2,100.35,57.70,42.28,37.82,36.15,34.80,33.74,32.93,31.94,31.56,32.14
1,SOFR,swaption,black,2024-03-05,2.00,3,96.27,55.55,40.93,36.76,35.21,33.97,33.01,32.28,31.41,31.13,31.73
2,SOFR,swaption,black,2024-03-05,0.25,3,92.74,55.24,42.59,39.23,38.02,37.06,36.32,35.76,35.07,34.73,35.06
3,SOFR,swaption,black,2024-03-05,1.00,2,100.35,57.70,42.28,37.82,36.15,34.80,33.74,32.93,31.94,31.56,32.14


In [9]:
IDVOLSKEW = 3
voldata = voldata.loc[[IDVOLSKEW],:]

# SABR Fit

In [10]:
SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
RELATIVE_STRIKE = 0

expry = 1
tenor = 2

freqswap = 4

Topt = expry
Tswap = Topt+tenor
fwdswap = calc_fwdswaprate(curves['discounts'], Topt, Tswap, freqswap=freqswap)

In [11]:
doSLIM = False
BETA = 0.25
F = fwdswap

In [12]:
volquote = voldata.query(f'model=="{QUOTE_STYLE}"').query(f'reference=="{SWAP_TYPE}"').query(f'date=="{DATE}"')#.query(f'expiration=="{expry}"').query(f'tenor=="{tenor}"')
idx = (volquote['expiration']==expry) & (volquote['tenor']==tenor)
volquote = volquote.loc[idx]
volquote.index = ['implied vol']

strikerange = np.array(volquote.columns[-11:].tolist())
vols = volquote[strikerange]
vols /= 100
strikes = fwdswap + strikerange/100/100
idstrike = np.where(strikerange==RELATIVE_STRIKE)[0][0]

idstrikeATM = np.where(strikerange==0)[0][0]

if QUOTE_STYLE == 'normal':
    vols /= 100 * fwdrate

capvol = curves.loc[Topt,'fwd vols']

strikeATM = strikes[idstrikeATM]
volATM = vols.iloc[0,idstrikeATM]

In [13]:
period_fwd = curves.index.get_loc(Topt)
period_swap = curves.index.get_loc(Tswap)+1
step = round(freqcurve/freqswap)

discount = curves['discounts'].iloc[period_fwd+step : period_swap : step].sum()/freqswap
blacks_quotes = vols.copy()
blacks_quotes.loc['strike'] = strikes
blacks_quotes = blacks_quotes.loc[['strike','implied vol']]

blacks_quotes.style.format('{:.4f}')

,-300,-200,-100,-50,-25,0,25,50,100,200,300
strike,0.0067,0.0167,0.0267,0.0317,0.0342,0.0367,0.0392,0.0417,0.0467,0.0567,0.0667
implied vol,1.0035,0.5770,0.4228,0.3782,0.3615,0.3480,0.3374,0.3293,0.3194,0.3156,0.3214


#### Limit SABR fit to quotes within +/- 200bps

In [14]:
vols = vols.loc[['implied vol'],-200:200]
strikes = strikes[1:-1]

In [15]:
def obj_fun(xargs):
    nu = xargs[0]
    rho = xargs[1]
    alpha = xargs[2]
    
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr(BETA,nu,rho,alpha,F,strike,Topt)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error


def obj_fun_slim(xargs):
    nu = xargs[0]
    rho = xargs[1]
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr_slim(BETA,nu,rho,F,strike,Topt,volATM)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error

In [16]:
if not doSLIM:
    x0 = np.array([.6,0,.1])
    fun = obj_fun
else:
    fun = obj_fun_slim
    x0 = np.array([.6,0,.1])

optim = minimize(fun,x0)
xstar = optim.x
nustar = xstar[0]
rhostar = xstar[1]

if doSLIM:
    alphastar = solve_alpha(BETA,nustar,rhostar,Topt,volATM,F)
    ivolSABR = sabr_slim(BETA,nustar,rhostar,F,strikes,Topt,volATM)
else:
    alphastar = xstar[2]
    ivolSABR = sabr(BETA,nustar,rhostar,alphastar,F,strikes,Topt)
    
error = optim.fun

/Users/markhendricks/Projects/finm-fiderivs-2024/build_data/../cmds/volskew.py:76: RuntimeWarning: invalid value encountered in scalar divide
  sigmaB = (NUM/DEN) * (z/chi(z,rho))
/Users/markhendricks/Projects/finm-fiderivs-2024/build_data/../cmds/volskew.py:76: RuntimeWarning: invalid value encountered in divide
  sigmaB = (NUM/DEN) * (z/chi(z,rho))


In [17]:
sabrparams = pd.DataFrame([BETA,alphastar,nustar,rhostar],index=['beta','alpha','nu','rho'],columns=['estimate']).style.format('{:.4f}')
sabrparams.index.name = 'parameter'

# Vol Path

Picture of Vol Path for ED futures is a screenshot from `W.4.X.`.

### Save Data

In [18]:
if SAVEDATA:    
    with pd.ExcelWriter(FILEOUT) as writer:  
        curves.to_excel(writer, sheet_name= f'rate curves', index=True)
        ratetree.to_excel(writer, sheet_name= f'rate tree', index=True)
        voldata.to_excel(writer, sheet_name= 'bloomberg vcub', index=False)
        sabrparams.to_excel(writer, sheet_name='sabr params', index=True)